In [2]:
import torch

import numpy as np

from tqdm import tqdm

from torch import nn

# Encodes categorical labels into numerical format (used for label preprocessing)
from sklearn.preprocessing import LabelEncoder

# Calculates the accuracy of a classification model (used for model evaluation)
from sklearn.metrics import accuracy_score

# Defines a custom dataset class for PyTorch (used for handling data)
from torch.utils.data import Dataset

# Creates a DataLoader for efficient batch processing in PyTorch (used for data loading)
from torch.utils.data import DataLoader

# Splits a dataset into training and validation sets (used for data splitting)
from torch.utils.data import random_split

# Represents a multi-dimensional matrix in PyTorch (used for tensor manipulation)
from torch import Tensor

# Implements a linear layer in a neural network (used for defining neural network architecture)
from torch.nn import Linear

# Applies rectified linear unit (ReLU) activation function (used for introducing non-linearity)
from torch.nn import ReLU

# Applies sigmoid activation function (used for binary classification output)
from torch.nn import Sigmoid

# Base class for all neural network modules in PyTorch (used for creating custom models)
from torch.nn import Module

# Stochastic Gradient Descent optimizer (used for model optimization during training)
from torch.optim import SGD

# Binary Cross Entropy Loss function (used for binary classification problems)
from torch.nn import BCELoss

# Initializes weights using Kaiming uniform initialization (used for weight initialization)
from torch.nn.init import kaiming_uniform_

# Initializes weights using Xavier (Glorot) uniform initialization (used for weight initialization)
from torch.nn.init import xavier_uniform_

e:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import pickle
with open('word_sequences.pkl', 'rb') as file:
    word_sequences = pickle.load(file)

with open('./pickles/char_sequences_without_tashkeel.pkl', 'rb') as file:
    char_sequences = pickle.load(file)

with open('./pickles/tashkeel_sequences.pkl', 'rb') as file:
    labels = pickle.load(file)

print(len(labels))

55618


In [4]:
# dataset definition
# A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.
class Dataset(Dataset):
    # load the dataset
    # The __init__ function is run once when instantiating the Dataset object
    def __init__(self):
   
        self.x = torch.tensor(char_sequences)
        self.y = torch.tensor(labels)
        
    # number of rows in the dataset
    # The __len__ function returns the number of samples in our dataset.
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.x))
        train_size = len(self.x) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])


# # prepare the dataset
# def prepare_data():
#     # load the dataset
#     dataset = CSVDataset()
#     # calculate split
#     train, test = dataset.get_splits()
#     # prepare data loaders
#     # The Dataset retrieves our dataset’s features and labels one sample at a time.
#     # While training a model, we typically want to pass samples in “minibatches”,
#     # reshuffle the data at every epoch to reduce model overfitting,
#     train_dl = DataLoader(train, batch_size=32, shuffle=True)
#     test_dl = DataLoader(test, batch_size=1024, shuffle=False)
#     return dataset.encoding_mapping, train_dl, test_dl

In [5]:
#convert labels to numpy array
print(len(labels[3]))
print(len(char_sequences[3]))
train_ds = Dataset()
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)

dg = iter(train_dl)
X1, Y1 = next(dg)
X2, Y2 = next(dg)
print(Y1.shape, X1.shape, Y2.shape, X2.shape)
print(X1[0][:], "\n", Y1[0][:])

2592
2592


torch.Size([64, 2592]) torch.Size([64, 2592]) torch.Size([64, 2592]) torch.Size([64, 2592])
tensor([12,  5,  1,  ...,  0,  0,  0], dtype=torch.int32) 
 tensor([3, 5, 2,  ..., 0, 0, 0], dtype=torch.int32)


MODEL

In [6]:
class Char_model(nn.Module):
  def __init__(self, vocab_size=39, embedding_dim=50, hidden_size=50, n_classes=17):
    """
    The constructor of our NER model
    Inputs:
    - vacab_size: the number of unique words
    - embedding_dim: the embedding dimension
    - n_classes: the number of final classes (tags)
    """
    super(Char_model, self).__init__()
    ####################### TODO: Create the layers of your model #######################################
    # (1) Create the embedding layer
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    # (2) Create an LSTM layer with hidden size = hidden_size and batch_first = True
    self.lstm = nn.LSTM(hidden_size, n_classes, batch_first=True)
    # batch_first makes the input and output tensors to be of shape (batch_size, seq_length, hidden_size)

    #####################################################################################################

  def forward(self, sentences):
    """
    This function does the forward pass of our model
    Inputs:
    - sentences: tensor of shape (batch_size, max_length)

    Returns:
    - final_output: tensor of shape (batch_size, max_length, n_classes)
    """

    final_output = None
    ######################### TODO: implement the forward pass ####################################
    final_output = self.embedding(sentences) 
    final_output, _ = self.lstm(final_output)   

    ###############################################################################################
    return final_output

In [7]:
model = Char_model()
print(model)

Char_model(
  (embedding): Embedding(38, 50)
  (lstm): LSTM(50, 17, batch_first=True)
)


# Training

In [8]:



def train(model, train_dataset, batch_size=128, epochs=5, learning_rate=0.01):
  """
  This function implements the training logic
  Inputs:
  - model: the model ot be trained
  - train_dataset: the training set of type NERDataset
  - batch_size: integer represents the number of examples per step
  - epochs: integer represents the total number of epochs (full training pass)
  - learning_rate: the learning rate to be used by the optimizer
  """
  
  # ana balef 3l dataset kolaha epoch mara
  # wl batch_size byb2a h3ml kam batch fe kol epoch
  # y3ny mthln, el data set 500, epoch 10, yb2a h3ady 3l 500 dol 10 marat
  # lw batch_size 5, yb2a el epoch hyb2a fyha 100 batch
  # lw batch_size 9, yb2a el epoch hyb2a fyha 12 batch, 11 menhom 9, wl 2a5eer 1

  # (1) create the dataloader of the training set (make the shuffle=True)
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  # (2) make the criterion cross entropy loss
  criterion = nn.CrossEntropyLoss()

  # (3) create the optimizer (Adam)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # GPU configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()

  for epoch_num in range(epochs):
    total_acc_train = 0
    total_loss_train = 0

    for train_input, train_label in tqdm(train_dataloader):
      
      # .to dh 3lshan an2el el 7aga lel GPU

      # (4) move the train input to the device
      train_label = train_label.long().to(device)

      # (5) move the train label to the device
      train_input = train_input.long().to(device)


      # (6) do the forward pass
      output = model(train_input)
      
      # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
      batch_loss = criterion(output.reshape(-1, 17), train_label.reshape(-1))
  
      # (8) append the batch loss to the total_loss_train
      total_loss_train += batch_loss.item()
      
      # (9) calculate the batch accuracy (just add the number of correct predictions)
      acc = (output.argmax(dim=2) == train_label).sum().item()
      total_acc_train += acc

      # (10) zero your gradients
      optimizer.zero_grad()
      

      # (11) do the backward pass
      batch_loss.backward()

      # (12) update the weights with your optimizer
      optimizer.step()
      
    # epoch loss
    epoch_loss = total_loss_train / len(train_dataset)

    # (13) calculate the accuracy
    epoch_acc = total_acc_train / (len(train_dataset) * len(train_dataset[0][0]))
    # ba2sem 3la 3adad el kalemat fy kol el gomal 
    # kol gomla asln fyha 104 kelma, fa badrab dh fy 3adad el gomal bs

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
        | Train Accuracy: {epoch_acc}\n')

  ##############################################################################################################

In [9]:
train_dataset = Dataset()
train(model, train_dataset)

  0%|          | 0/435 [00:00<?, ?it/s]


IndexError: index out of range in self

: 

# Evaluation

In [ ]:
def evaluate(model, test_dataset, batch_size=512):
  """
  This function takes a NER model and evaluates its performance (accuracy) on a test data
  Inputs:
  - model: a NER model
  - test_dataset: dataset of type NERDataset
  """
  ########################### TODO: Replace the Nones in the following code ##########################

  # (1) create the test data loader
  test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

  # GPU Configuration
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  if use_cuda:
    model = model.cuda()

  total_acc_test = 0
  
  # (2) disable gradients
  with torch.no_grad():
    # 3mlna disable 3lshan e7na bn-predict (aw evaluate y3ny) b2a dlw2ty, msh bn-train

    for test_input, test_label in tqdm(test_dataloader):
      # (3) move the test input to the device
      test_label = test_label.to(device)

      # (4) move the test label to the device
      test_input = test_input.to(device)
      # brdo the comments should be reversed 

      # (5) do the forward pass
      output = model(test_input)

      # accuracy calculation (just add the correct predicted items to total_acc_test)
      acc = (output.argmax(dim=2) == test_label).sum().item()
      total_acc_test += acc
    
    # (6) calculate the over all accuracy
    total_acc_test /= (len(test_dataset) * len(test_dataset[0][0]))
  ##################################################################################################

  
  print(f'\nTest Accuracy: {total_acc_test}')